In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='3' 

In [2]:
import fastai
import ffmpeg
from fastai import *
from fastai.vision import *
from fastai.callbacks.tensorboard import *
from fastai.vision.gan import *
from fasterai.dataset import *
from fasterai.visualize import *
from fasterai.loss import *
from fasterai.filters import *
from fasterai.generators import *
from pathlib import Path
from itertools import repeat
from IPython.display import HTML, display
plt.style.use('dark_background')
torch.backends.cudnn.benchmark=True

In [3]:
#Adjust render_factor (int) if image doesn't look quite right (max 64 on 11GB GPU).  The default here works for most photos.  
#It literally just is a number multiplied by 16 to get the square render resolution.  
#Note that this doesn't affect the resolution of the final output- the output is the same resolution as the input.
#Example:  render_factor=21 => color is rendered at 16x21 = 336x336 px.  
render_factor=25
root_folder =  Path('data/imagenet/ILSVRC/Data/CLS-LOC/bandw')
weights_name = 'ColorizeNew44_gen19205'
#weights_name = 'ColorizeNew32_gen'
nf_factor = 1.25

workfolder = Path('./video')
source_folder = workfolder/"source"
bwframes_root = workfolder/"bwframes"
colorframes_root = workfolder/"colorframes"
result_folder = workfolder/"result"
#Make source_url None to just read from source_path directly without modification
source_url = 'https://twitter.com/silentmoviegifs/status/1092793719173115905'
#source_url=None
source_name = 'video5.mp4'
source_path =  source_folder/source_name
bwframes_folder = bwframes_root/(source_path.stem)
colorframes_folder = colorframes_root/(source_path.stem)
result_path = result_folder/source_name

In [4]:
def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 40%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

In [5]:
def get_fps():
    probe = ffmpeg.probe(str(source_path))
    stream_data = next((stream for stream in probe['streams'] if stream['codec_type'] == 'video'), None)
    avg_frame_rate = stream_data['avg_frame_rate']
    print(avg_frame_rate)
    fps_num=avg_frame_rate.split("/")[0]
    fps_den = avg_frame_rate.rsplit("/")[1]
    return round(float(fps_num)/float(fps_den))

In [6]:
def purge_images(dir):
    for f in os.listdir(dir):
        if re.search('.*?\.jpg', f):
            os.remove(os.path.join(dir, f))

## Download Video (optional via setting source_url)

##### Specify media_url.  Many sources will work (YouTube, Imgur, Twitter, Reddit, etc). Complete list here:  https://rg3.github.io/youtube-dl/supportedsites.html .  The resulting file path can be used later.

In [7]:
if source_url is not None:
    if source_path.exists(): source_path.unlink()
    youtubdl_command = 'youtube-dl "' + source_url + '" -o "' + str(source_path) + '"'
    print(youtubdl_command)
    print('\n')
    output = Path(os.popen(youtubdl_command).read())
    print(str(output))

youtube-dl "https://twitter.com/silentmoviegifs/status/1092793719173115905" -o "video/source/video5.mp4"


[twitter] 1092793719173115905: Downloading webpage
[twitter:card] 1092793719173115905: Downloading webpage
[twitter:card] 1092793719173115905: Downloading guest token
[twitter:card] 1092793719173115905: Downloading JSON metadata
[download] Destination: video/source/video5.mp4

[download]   0.4% of 252.07KiB at 567.03KiB/s ETA 00:00
[download]   1.2% of 252.07KiB at  1.55MiB/s ETA 00:00
[download]   2.8% of 252.07KiB at  3.37MiB/s ETA 00:00
[download]   6.0% of 252.07KiB at  6.52MiB/s ETA 00:00
[download]  12.3% of 252.07KiB at  5.96MiB/s ETA 00:00
[download]  25.0% of 252.07KiB at  3.37MiB/s ETA 00:00
[download]  50.4% of 252.07KiB at  3.60MiB/s ETA 00:00
[download] 100.0% of 252.07KiB at  5.21MiB/s ETA 00:00
[download] 100% of 252.07KiB in 00:00



## Extract Raw Frames

In [8]:
bwframe_path_template = str(bwframes_folder/'%5d.jpg')
bwframes_folder.mkdir(parents=True, exist_ok=True)
purge_images(bwframes_folder)
ffmpeg.input(str(source_path)).output(str(bwframe_path_template), format='image2', vcodec='mjpeg', qscale=0).run(capture_stdout=True)

(b'', None)

In [9]:
framecount = len(os.listdir(str(bwframes_folder)))

## DeOldify / Colorize

In [10]:
vis = get_colorize_visualizer(root_folder=root_folder, weights_name=weights_name, nf_factor=nf_factor, render_factor=render_factor)
#vis = get_colorize_visualizer(render_factor=render_factor)

/media/jason/Projects/Deep Learning/DeOldifyV2/DeOldify/fastai/data_block.py:414: UserWarning: Your training set is empty. Is this is by design, pass `ignore_empty=True` to remove this warning.
  warn("Your training set is empty. Is this is by design, pass `ignore_empty=True` to remove this warning.")
/media/jason/Projects/Deep Learning/DeOldifyV2/DeOldify/fastai/data_block.py:417: UserWarning: Your validation set is empty. Is this is by design, use `no_split()` 
                 or pass `ignore_empty=True` when labelling to remove this warning.
  or pass `ignore_empty=True` when labelling to remove this warning.""")


In [11]:
prog = 0
out = display(progress(0, 100), display_id=True)
colorframes_folder.mkdir(parents=True, exist_ok=True)
purge_images(colorframes_folder)

for img in os.listdir(str(bwframes_folder)):
    img_path = bwframes_folder/img
    if os.path.isfile(str(img_path)):
        color_image = vis.get_transformed_image(str(img_path), render_factor)
        color_image.save(str(colorframes_folder/img))
    prog += 1
    out.update(progress(prog, framecount))

## Build Video

In [12]:
colorframes_path_template = str(colorframes_folder/'%5d.jpg')
result_path.parent.mkdir(parents=True, exist_ok=True)

if result_path.exists(): result_path.unlink()

fps = get_fps()
print(fps)
ffmpeg.input(str(colorframes_path_template), format='image2', vcodec='mjpeg', framerate=str(fps)).output(str(result_path), crf=17, vcodec='libx264').run(capture_stdout=True)

71/7
10


(b'', None)